# **Analytical solution for 1D continuous solute transport injection in porous media (without decay)**

The equation that defines 1D continuous solute transport injection, without decay, in porous media is the following,

$C(x, t) = \frac{C_{0}}{2}\left[ \operatorname{erfc}\!\left( \frac{x - vt}{\sqrt{4Dt}}\right) + \operatorname{exp}\!\left(\frac{vx}{D}\right) \operatorname{erfc}\!\left( \frac{x + vt}{\sqrt{4Dt}}\right)\right]$

if $\frac{vx}{D} > 100$, we can approximate the solution as $C(x, t) = \frac{C_{0}}{2}\left[ \operatorname{erfc}\!\left( \frac{x - vt}{\sqrt{4Dt}}\right)\right]$

**References:**
- Ogata, A., & Banks, R. B. (1961). A solution of the differential equation of longitudinal dispersion in porous media: fluid movement in earth materials. US Government Printing Office.
- Bear, J., & Cheng, A. H. D. (2010). Modeling groundwater flow and contaminant transport (Vol. 23, p. 834). Dordrecht: Springer.
- Bear, J. (2012). Hydraulics of groundwater. Courier Corporation.

<br>

## **1. Loading the needed libraries**

In [1]:
#-- Check and install required packages if not already installed 
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed 
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")
if_require("numpy")
if_require("scipy")
if_require("os")

#-- Import the rest of libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import scipy as sp
import os

from IPython.display import display, clear_output, HTML
from ipywidgets import Layout, Output
from tkinter import Tk, filedialog

import warnings
warnings.filterwarnings("ignore")

#-- LaTeX configuration, if installed
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Computer Modern Roman"
# })

#-- Widgets parameters configuration
param_widgets = { 
                 'readout_format': ".2f", 
                 'style': {'description_width': '200px'}, 
                 'layout': widgets.Layout(width='500px')
                 }  

## **2. Coding the equations and plotting the results**

In [2]:
#--Dispersion coefficient
def dispersion(alpha: float, v: float) -> float:
    """Calculates the hydrodynamic dispersion coefficient.

    Parameters
    ----------
    alpha : float
        Dispersivity [L]
    v : float
        Groundwater velocity [L/T]

    Returns
    -------
    float
        Dispersion coefficient [L²/T]
    """
    
    D = alpha * v
    return D

#--Dimensionless parameters: Péclet number
def peclet_parameter(x: float, v: int, D: float) -> float:
    """Calculates the Péclet number.

    The Péclet number is a dimensionless parameter that compares
    advective transport to dispersive transport in porous media:
    - High Pe (≫1): advection dominates
    - Low Pe (≪1): dispersion dominates
    
    Parameters
    ----------
    x : float
        Position along the x axis [L]
    v : float
        velocity [L/T]
    D : float
        Dispersion coefficient [L²/T]

    Returns
    -------
    float
        Péclet value [-]
    """
    
    Pe = (v*x)/D
    
    return Pe


#--One-dimensional continuous injection equation         
def oneD_continous_injection(Ci: float, x: float | np.ndarray, v: float, t: float | np.ndarray, 
                             alpha: float, adv: bool) -> np.ndarray:
    """Calculates the concentration in a one-dimensional continous injection,
    with no adsorption or decay (Ogata & Banks, 1961).

    Parameters
    ----------
    Ci : float
        Initial concentration [M/V]
    x : float
        Position along the x axis [L]
    v : float
        Groundwater velocity [L/T]
    t : float
        Time
    alpha : float
        Dispersivity [L]
    adv: Boolean
        If False, the second term of the equation will be considered; 
        If True, the second term of the equation will be neglected.

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    #--Dispersion coefficient
    D = dispersion(alpha, v)
    
    #--Dimensionless transport parameters: Peclet number
    Pe = peclet_parameter(x, v, D)
    
    #--Transport equation 
    term1 = sp.special.erfc((x - (v*t))/(2*(np.sqrt(D*t))))
    term2_1 = np.exp(Pe)
    term2_2 = sp.special.erfc((x + (v*t)) / (2*(np.sqrt(D*t))))
       
    if adv == False:
        C = (Ci/2) * (term1 + (term2_1 * term2_2))
    else:
        C = (Ci/2) * (term1)
        
    return C

By varying the values of dispersivity with the slider, we can see when the second term of the equation can be neglected. Remember that changing the dispersivity values directly changes the hydrodynamic dispersion coefficient and, therefore, the Péclet number:

$Pe = \frac{vx}{D}$

In [3]:
def find_parameters(alpha):   
        t_values = [50, 100, 150]
        x_values = [50, 100, 150]
     
        c_space = [oneD_continous_injection(1, np.arange(0, 201), 1, t, alpha, False) for t in t_values]
        c_space_adv = [oneD_continous_injection(1, np.arange(0, 201), 1, t, alpha, True) for t in t_values]
        c_time = [oneD_continous_injection(1, x, 1, np.arange(0, 201), alpha, False) for x in x_values]
        c_time_adv = [oneD_continous_injection(1, x, 1, np.arange(0, 201), alpha, True) for x in x_values]
        
        #-- Plotting the results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        #--space
        ax1.plot(np.arange(0, 201), c_space[0], c="cornflowerblue", label="C(x, 50)")
        ax1.plot(np.arange(0, 201), c_space[1], c="limegreen", label="C(x, 100)")
        ax1.plot(np.arange(0, 201), c_space[2], c="tomato", label="C(0, 150)")
        ax1.plot(np.arange(0, 201), c_space_adv[0], ':', c="cornflowerblue", label="C(x, 50)—2nd term neglected", linewidth=3)
        ax1.plot(np.arange(0, 201), c_space_adv[1], ':', c="limegreen", label="C(x, 100)—2nd term neglected", linewidth=3)
        ax1.plot(np.arange(0, 201), c_space_adv[2], ':', c="tomato", label="C(0, 150)—2nd term neglected", linewidth=3)
        ax1.set_xlim(0, 201)
        ax1.set_ylim(0, 1)
        ax1.set_xlabel(r'x')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
        #--time
        ax2.plot(np.arange(0, 201), c_time[0], c="cornflowerblue", label="C(50, t)")
        ax2.plot(np.arange(0, 201), c_time[1], c="limegreen", label="C(100, t)")
        ax2.plot(np.arange(0, 201), c_time[2], c="tomato", label="C(150, t)")
        ax2.plot(np.arange(0, 201), c_time_adv[0], ':', c="cornflowerblue", label="C(50, t)—2nd term neglected", linewidth=3)
        ax2.plot(np.arange(0, 201), c_time_adv[1], ':', c="limegreen", label="C(100, t)—2nd term neglected", linewidth=3)
        ax2.plot(np.arange(0, 201), c_time_adv[2], ':', c="tomato", label="C(150, t)—2nd term neglected", linewidth=3)
        ax2.set_xlim(0, 201)
        ax2.set_ylim(0, 1)
        ax2.set_xlabel(r't')
        ax2.set_ylabel('Concentration')
        ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax2.get_position()
        ax2.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
        fig.tight_layout()
        plt.show()

sliders = [
        widgets.FloatSlider(value=1, min=0.3, max=10, step=0.1, description='Dispersivity [m]', **param_widgets)
]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters, {
        'alpha': sliders[0]
        })

display(ui, out)

Output()

Next, we can check when the contaminant first reaches an observation point located 125 metres from the source, and determine the maximum concentration there.

In [4]:
def find_parameters_v2(Ci, t, alpha):   
       
        c_space = oneD_continous_injection(Ci, np.arange(0, 201), 1, t, alpha, False)
        
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
        
        #--space
        ax1.plot(np.arange(0, 201), c_space, c="cornflowerblue", label="Concentration")
        ax1.set_xlim(0, 201)
        ax1.set_ylim(0, 101)
        ax1.set_xlabel(r'x')
        ax1.set_ylabel('Concentration')
        ax1.vlines(x=125, ymin=0, ymax=100, linestyle="dotted", color="k", label="Observation point")
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
sliders = [
        widgets.FloatSlider(value=40, min=20, max=100, step=20, description='Concentration [mg/L]', **param_widgets),
        widgets.FloatSlider(value=10, min=0, max=200, step=1, description='Time [d]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.3, max=10, step=0.1, description='Dispersivity [m]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters_v2, {
        'Ci': sliders[0],
        't': sliders[1],
        'alpha': sliders[2],
        })

display(ui, out)

Output()